# Final Project University of Nottingham Final Exam Scheduling

### Team members: Kien Le, Jenny Nguyen, Dionne Phan
### Instructor: Dr. Bonnifonte

## I - Introduction and motivation

During final exam season (including the time when we wrote this report), it is common to see students expressing stress and frustration about their tightly packed schedules. As one student shared, "My exam schedule is terrible. I have to take three exams in a row from morning to night, and then turn in my final project at 10 a.m. the next morning" (Kien during his sophomore year). Experiences like this are common at Denison, and they highlight a central concern: **why do exam schedules end up being so stressful, and could they be designed more fairly?**

This project addresses that concern by examining the principles behind final exam schedules and the challenges that limit how balanced they can be. Our goal is to understand the basic structure of the exam scheduling problem and determine whether seemingly "bad" schedules are the result of randomness, unavoidable constraints, or flaws in the scheduling model itself. While we initially hoped to analyze Denison data, that information was not available (and we know that another group had already conducted a similar project). Instead, we used the **University of Nottingham’s final exam schedule** as a representative dataset, since it reflects the typical constraints faced by universities—such as exam room capacity, overlapping student enrollments, departmental requirements, fixed exam periods, and other complex considerations.

From this dataset, we set out to answer four central research questions:

1. **What is the definition of a "good" exam schedule for students?**
2. **How can an exam schedule be constructed in a systematic and principled way?**
3. **What are the key bottlenecks or limiting factors that make exam scheduling difficult?**
4. **If we build a working scheduling model, how can it be applied or adapted to Denison’s exam system?**

Understanding these questions is valuable not only for students, but also for administrators responsible for producing exam schedules. Insights from this analysis may reveal whether better scheduling algorithms could lead to a better exam experiences, or whether certain bottlenecks are simply unavoidable.

This report is organized as follows. Section 2 presents the preliminaries, including our data sources, data-wrangling process, and the overall setup of the problem. Section 3 details the modeling steps we used to construct and evaluate exam schedules, followed by an analysis of the results. Finally, Section 4 discusses how our model could be applied to Denison’s exam scheduling system and concludes with the broader implications of our findings.

## II - Preliminaries

In this section, we introduce the data sources and structure of the dataset, outline the key objectives guiding our exam scheduling model, and describe our preprocessing steps and setup required for model construction,.

### a. Data Source and Structure

The [exam timetabling dataset](https://people.cs.nott.ac.uk/pszrq/data.htm) was obtained from the University of Nottingham and is publicly available for research purposes. It was originally introduced by Burke, Newall, and Weare (1996) in their foundational study on university examination scheduling and has since become a standard benchmark within the exam timetabling community. The dataset represents real scheduling data of the 1994 - 1995 academic year at the University of Nottingham and includes detailed information on students, courses, exams, and enrolments. In total, the dataset contains 800 exams across 46 departments, 7,896 students, and 33,997 enrollments—equivalent to an average of 4.3 exams per student and 42.5 students per exam. A key feature of the dataset is its global room-capacity constraint: in any given timeslot, the total number of students taking exams cannot exceed 1,550. The primary objective defined in the original study is to minimize the number of students assigned to two consecutive exams on the same day. A later extension expands the objective to also avoid consecutive overnight exams (Burke & Carter, 1998).

Because it is based on real institutional data, incorporates practical scheduling constraints, and defines clear optimization objectives, the University of Nottingham dataset provides a rigorous, transparent, and reproducible benchmark for evaluating exam scheduling models. The dataset is provided in text format and consists of four main files:

1. `students`: This file contains information on student enrollment in courses. It has two columns:
* Student code: a 10-character unique identifier for each student.
* Course code: the code of the course the student is enrolled in.

1. `exams`: This file provides details about each exam and consists of four columns:
* Exam code: a unique identifier for the exam.
* Course name: the name of the course associated with the exam.
* Exam duration: the length of the exam in minutes.
* Department code: the code of the department responsible for the exam.

1. `enrolments`: This file links the students and exams datasets. It contains two columns:
* Student code: the unique identifier for a student.
* Exam code: the unique identifier for an exam in which the student is enrolled.

1. `data`: This file is not a dataset but specifies the necessary conditions and constraints for course scheduling, such as room capacities, allowable time slots, and other parameters and special constraints required for constructing the timetable.

### b. Model objectives 

Our model is designed to achieve the following objectives:

1. **Construct a feasible exam schedule** that satisfies all required constraints, including room capacity limits, departmental requirements, and student enrollment.

2. **Minimize the number of students assigned to two consecutive exams on the same day**.

3. **Minimize overnight consecutive exams**, ensuring that students have adequate time to rest and prepare between assessment periods.

Given these three objectives, our modeling procedure is designed to achieve them step by step. This structured approach not only guides the construction of a feasible timetable but also helps identify the key bottlenecks and challenges that arise during exam scheduling. Taken together, these objectives aim to produce an exam schedule that is both logistically sound and student-centered, balancing operational feasibility with consideration for student well-being.

In [2]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, CustomJS, FixedTicker, LabelSet
from bokeh.layouts import column
output_notebook()

Loading BokehJS ...

In [4]:
ROOT = Path().resolve().parent

# load data
enrolments = pd.read_csv(ROOT / "converted-data" / "enrolments.csv")
exams = pd.read_csv(ROOT / "converted-data" / "exams.csv")
students = pd.read_csv(ROOT / "converted-data" / "students.csv")
coincidences = pd.read_csv(ROOT / "converted-data" / "coincidences.csv", index_col=0)
dates = pd.read_csv(ROOT / "converted-data" / "dates.csv")
times_expanded = pd.read_csv(ROOT / "converted-data" / "times_expanded.csv")
rooms = pd.read_csv(ROOT / "converted-data" / "rooms.csv")

In [5]:
# convert exam duration from hh:mm to minutes
def parse_duration(time_str):
    h, m = time_str.split(":")
    return int(h) * 60 + int(m)

# exams
exams["duration_minutes"] = exams["duration"].apply(parse_duration)

# exam list and duration dict
exam_list = exams["exam"].unique().tolist()
duration = exams.set_index("exam")["duration_minutes"].to_dict()

# give each row a unique timeslot index
times_expanded["timeslot"] = range(1, len(times_expanded) + 1)

# convert hours -> minutes
times_expanded["slot_minutes"] = times_expanded["duration_hours"] * 60
timeslots = times_expanded["timeslot"].tolist()
slot_length = times_expanded.set_index("timeslot")["slot_minutes"].to_dict()

# student -> list of exams
students_exams = enrolments.groupby("student")["exam"].apply(list).to_dict()

# exams -> number of students
exam_student_count = enrolments.groupby("exam")["student"].nunique().to_dict()

# define list of room and capacity of each room
room_list = rooms["room"].tolist()
cap = rooms.set_index("room")["capacity"].to_dict()

# count unique students per exam
exam_counts = enrolments.groupby("exam")["student"].nunique().sort_values(ascending=False)

size = exam_counts.to_dict()

# Define fixed exam slot times
time_order = ["9:00", "13:30", "16:30"]

# Map time strings to numeric Y values (for plotting)
time_to_y = {t: i + 1 for i, t in enumerate(time_order)}
day_to_x = {"Mon": 1, "Tue": 2, "Wed": 3, "Thu": 4, "Fri": 5, "Sat": 6}


In [6]:
def plot_source(schedule_df):
    """
    Convert the given schedule to plot source
    schedule_df: the schedule df with the columns [exam, day, week, start_time, duration_min, timeslot]
    Output: plot source for plotting
    """
    x_coords, y_coords, exam_texts, week_list = [], [], [], []

    for _, row in schedule_df.iterrows():
        day = row["day"]
        time = row["start_time"]
        week = row["week"]

        # numeric y coordinate
        y_val = time_to_y[time] + 4 * (week - 1)

        x_coords.append(day_to_x[day])
        y_coords.append(y_val)
        exam_texts.append(row["exam"])
        week_list.append(week)
    student_counts = [exam_student_count[e] for e in exam_texts]

    # Combine exams in same cell
    df_vis = pd.DataFrame({
        "day": x_coords,
        "y": y_coords,
        "week": week_list,
        "exam": exam_texts,
        "students": student_counts
    })

    df_grouped = (
        df_vis.groupby(["day", "y", "week"])
        .agg({
            "exam": lambda x: list(x),
            "students": "sum"
        })
        .reset_index()
    )
    df_grouped["exam_count"] = df_grouped["exam"].apply(len)

    df_grouped["label"] = df_grouped.apply(
        lambda r: f"{r['exam_count']} exams \n ({r['students']} students)",
        axis=1
    )
    source = ColumnDataSource(df_grouped)
    return source

def make_plot (source, plot_name):
    """
    Plot the given plot source
    source: Plot source
    plot_name: Plot name
    """
    p = figure(
        x_range=(0.5, 6.5),
        y_range=[8, 0],
        height=600,
        title=plot_name,
        tools="tap",
    )
    p.yaxis.ticker = FixedTicker(ticks=[1, 2, 3, 5, 6, 7])
    p.ygrid.ticker = FixedTicker(ticks=[1, 2, 3, 5, 6, 7])
    p.yaxis.major_label_overrides = {1: "9:00", 2: "13:30", 3: "16:30", 5: "9:00", 6: "13:30", 7: "16:30"}
    p.xaxis.ticker = [1, 2, 3, 4, 5, 6]
    p.xaxis.major_label_overrides = {1:"Mon", 2:"Tue", 3:"Wed", 4:"Thu", 5:"Fri", 6:"Sat"}
    p.xaxis.axis_label = "Day"
    p.yaxis.axis_label = "Start Time"
    p.xaxis.major_label_orientation = 1.2

    p.rect(
        x="day",
        y="y",
        width=0.9,
        height=0.8,
        source=source,
        fill_color="skyblue",
        line_color="black",
    )

    p.text(
        x="day",
        y="y",
        text="label",
        source=source,
        text_align="center",
        text_baseline="middle",
        text_font_size="8pt",
    )

    # Callback
    callback = CustomJS(
        args=dict(source=source),
        code="""
        const data = source.data;
        const idx = source.selected.indices[0];
        if(idx !== undefined){
            const day = data['day'][idx];
            const week = data['week'][idx];
            const y = data['y'][idx];
            const exams = data['exam'][idx];

            alert(
                'Day: ' + day +
                '\\nWeek: ' + week +
                '\\nTime slot: ' + y +
                '\\nExams: ' + exams
            );
        }
    """,
    )

    source.selected.js_on_change("indices", callback)

    show(p)


### c. Data Preprocessing and Setup

Before building the model, we performed data wrangling and set up the variables that will be used in the subsequent modeling steps. The key steps we did on data wrangling are as follows:

1. Convert dataset formats: The original text files were transformed into CSV format to facilitate easier data access and manipulation.
2. Merge datasets and extract variables: Relevant information from the students, exams, enrolments, and data files was combined, and the variables required for modeling were extracted.

The variables will be used in the next modeling section are defined as follows:

* `exam_list` *(List)*: A list containing all exam codes.
* `timeslots` *(List)*: A list of available timeslots, enumerated from 1 to 32.
* `students_exams` *(Dictionary)*: A mapping where each key is a student code, and the corresponding value is the list of exams that the student is required to take, drawn from the `exam_list`.
* `durations` *(Dictionary)*: A mapping where each key is an exam code, and the corresponding value is the exam length (in minutes).
* `size` *(Dictionary)*: A mapping where each key is an exam code, and the corresponding value is the number of students enrolls in that exam.

In addition, visualization methods were also designed to display the results of the generated schedules. These visualizations help to understand the structure and quality of the solutions and support easier interpretation of the outcomes.

## III - Modelling

In this section, we will iteratively build the exam scheduling models, gradually adding constraints from one model to the next. 

### Model 1: A basic feasible exam timetabling model
This initial model, while simple and not fully constrained, is presented in detail to establish a foundation for understanding the subsequent models. Thus, the repetitive and tedious variables and constraints in the later models will be omitted. We will instead focus on explaining the key constraints and modifications introduced at each stage.

We first define the following sets:

- $C$: the set of exams.
- $T$: the set of available timeslots.
- $S$: the set of students.
- $E_s \subseteq C$: the set of exams taken by student $s \in S$.

We will next define our decision variables $x_{i,t}$ for $i \in C$ and $t \in T$ as

$$
x_{i,t} =
\begin{cases}
1, & \text{if exam } i \text{ is scheduled in timeslot } t, \\
0, & \text{otherwise}.
\end{cases}
$$

For this model, we apply the following constraints:

1. Every exam is assigned to one and only one timeslot, which means
$$
\sum_{t \in T} x_{i,t} = 1 \qquad \forall i \in C.
$$

2. For every student $s$ and for every pair of exams $(i, j \in E_s)$ with $(i \neq j)$, they cannot be assigned to the same timeslot:
$$
x_{i,t} + x_{j,t} \le 1
\qquad \forall s \in S,\; \forall i \neq j \in E_s,\; \forall t \in T.
$$

3. Exam duration must not exceed timeslot length. Let $\text{dur}_i$ be the duration of exam $i$, and let $\text{len}_t$ be the allowable duration of timeslot $t$. If an exam is too long to fit into a timeslot, the assignment is forbidden:

$$
x_{i,t} = 0
\qquad \forall i \in C,\; \forall t \in T \text{ such that } \text{dur}_i > \text{len}_t.
$$

For this model, our aim is just to obtain a feasible schedule. Thus, we may use a dummy objective:

$$
\min 0
$$

or equivalently:

$$
\text{Find any feasible assignment satisfying all constraints.}
$$

The implementation of this model is the following

In [7]:
# MODEL 1
model1 = gp.Model("Basic feasible exam timetabling model")
model1.setParam("OutputFlag", 0)

# Decision vars x[e,t]
x = model1.addVars(exam_list, timeslots, vtype=GRB.BINARY, name="x")

# 1. Each exam must be scheduled exactly once
for e in exam_list:
    model1.addConstr(
        gp.quicksum(x[e, t] for t in timeslots) == 1, name=f"assign_once_{e}"
    )

# 2. No student may have two exams in the same timeslot
for s, exam_list_s in students_exams.items():
    if len(exam_list_s) > 1:
        for t in timeslots:
            for i in range(len(exam_list_s)):
                for j in range(i + 1, len(exam_list_s)):
                    e1 = exam_list_s[i]
                    e2 = exam_list_s[j]
                    model1.addConstr(
                        x[e1, t] + x[e2, t] <= 1, name=f"no_overlap_{s}_{e1}_{e2}_{t}"
                    )

# 3. Exam duration <= timeslot duration
for e in exam_list:
    for t in timeslots:
        if duration[e] > slot_length[t]:
            model1.addConstr(x[e, t] == 0, name=f"forbidden_duration_{e}_{t}")

# Objective: Feasible schedule
model1.setObjective(0, GRB.MINIMIZE)

# Solve model
model1.optimize()

Set parameter Username
Set parameter LicenseID to value 2711254
Academic license - for non-commercial use only - expires 2026-09-21


In [8]:
def get_schedule_model1(x):
    """
    Extracts the exam schedule from the solution of Model 1 and converts it into a DataFrame
    x: Decision variables
    Output: A table containing the scheduled exams
    """
    schedule_rows = []

    for e in exam_list:
        for t in timeslots:
            if x[e, t].X > 0.5:
                row = times_expanded[times_expanded["timeslot"] == t].iloc[0]
                schedule_rows.append(
                    {
                        "exam": e,
                        "day": row["day"],
                        "week": row["week"],
                        "start_time": row["start_time"],
                        "duration_min": row["slot_minutes"],
                        "timeslot": t,
                    }
                )

    # Turn into a DataFrame
    schedule_df = pd.DataFrame(schedule_rows)
    schedule_df.sort_values(["timeslot"], inplace=True)
    return schedule_df

In [ ]:
# The exam timetable for Model 1
schedule_df = get_schedule_model1(x)
source1 = plot_source(schedule_df)
make_plot(source1, "Exam Timetable(Model 1)")

### Model 2: Exam timetabling with room assignment

This model will next introduce the room size constraints for the model. That is, the number of students in each exam must not exceed the capacity of the room that one is assigned for. To incorporate room capacity constraints into the model, we introduce a new set $R$, representing the set of available exam rooms. For each room $r \in R$, we define $\text{cap}_r$ as its capacity. Similarly, for each exam $i \in C$, we denote $\text{size}_i$ as the number of students enrolled in that exam.

We also introduce a new set of binary decision variables $y_{i,r}$ for each exam $i \in C$ and room $r \in R$, defined as:

$$
y_{i,r} =
\begin{cases}
1 & \text{if exam } i \text{ is assigned to room } r, \
0 & \text{otherwise.}
\end{cases}
$$

The model is then subject to the following constraints:

1. Each exam must be assigned to exactly one room:
$$
\sum_{r \in R} y_{i,r} = 1, \quad \forall i \in C
$$

2. Room capacity constraint: An exam cannot be assigned to a room if the room’s capacity is smaller than the number of students enrolled in the exam:
$$
y_{i,r} = 0, \quad \forall i \in C, ; \forall r \in R \text{ such that } \text{cap}_r < \text{size}_i
$$

3. A room cannot host more than one exam in the same timeslot:
$$
   x_{i,t} + x_{j,t} + y_{i,r} + y_{j,r} \le 3,
   \quad \forall r \in R, ; \forall t \in T, ; \forall i < j
$$

The implementation of this model is the following

In [14]:
max_room_capacity = max(cap.values())  # largest room

oversized_exams = [e for e in exam_list if size[e] > max_room_capacity]

print("Exams that cannot fit in any room:", oversized_exams)

# Remove from exam list
exam_list = [e for e in exam_list if e not in oversized_exams]

# Also remove from students_exams
for s in students_exams:
    students_exams[s] = [e for e in students_exams[s] if e in exam_list]

Exams that cannot fit in any room: []


In [15]:
# MODEL 2

model2 = gp.Model("Exam timetabling with room assignment")

x_allowed = [(e, t) for e in exam_list for t in slot_length if duration[e] <= slot_length[t]]
y_allowed = [(e, r) for e in exam_list for r in cap if size[e] <= cap[r]]
x = model2.addVars(x_allowed, vtype=GRB.BINARY, name="x")  # exam <-> timeslot
y = model2.addVars(y_allowed, vtype=GRB.BINARY, name="y")  # exam <-> room

# 1. Each exam assigned exactly one timeslot
# for s, exams_s in students_exams.items():
#     for t in timeslots:
#         relevant_exams = [e for e in exams_s if (e, t) in x]
#         if len(relevant_exams) > 1:
#             model2.addConstr(
#                 gp.quicksum(x[e, t] for e in relevant_exams) <= 1
#             )


# 2. Each exam assigned exactly one room
# for e in exam_list:
#     allowed_rooms = [r for r in room_list if (e, r) in y_allowed]
#     model2.addConstr(
#         gp.quicksum(y[e, r] for r in allowed_rooms) == 1,
#         name=f"room_once_{e}"
#     )

# 3. No student overlap constraint
# for s, exams_s in students_exams.items():
#     for t in timeslots:
#         # Only include allowed exam-timeslot pairs
#         relevant_exams = [e for e in exams_s if (e, t) in x_allowed]
#         if len(relevant_exams) > 1:
#             model2.addConstr(
#                 gp.quicksum(x[e, t] for e in relevant_exams) <= 1,
#                 name=f"no_overlap_student_{s}_{t}"
#             )

# # 4. Room capacity constraint
# for e in exam_list:
#     for r in room_list:
#         if size[e] > cap[r]:  # cannot fit
#             model2.addConstr(y[e, r] == 0, name=f"cap_forbidden_{e}_{r}")

# # 5. Exam duration must fit slot length
# for e in exam_list:
#     for t in timeslots:
#         if duration[e] > slot_length[t]:
#             model2.addConstr(x[e, t] == 0, name=f"duration_forbidden_{e}_{t}")

# 6. A room cannot host 2 exams in the same slot
# for r in room_list:
#     for t in timeslots:
#         exams_possible = [e for e in exam_list if (e, t) in x and (e, r) in y]
#         if len(exams_possible) > 0:
#             model2.addConstr(
#                 gp.quicksum(x[e, t] * y[e, r] for e in exams_possible) <= 1,
#                 name=f"room_conflict_r{r}_t{t}"
#             )


# Objective = feasibility
model2.setObjective(0, GRB.MINIMIZE)

model2.optimize()

# print("Model 2: Exam timetabling with room assignment \n")
# for e in exam_list:
#     assigned_t = [t for t in timeslots if x[e, t].X > 0.5][0]
#     assigned_r = [r for r in room_list if y[e, r].X > 0.5][0]
#     print(f"Exam {e} -> Timeslot {assigned_t}, Room {assigned_r}")


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 34217 columns and 0 nonzeros
Model fingerprint: 0x39656b54
Variable types: 0 continuous, 34217 integer (34217 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


## Model 3: Exam timetabling with room merging

## Sets
$$
\begin{aligned}
C &: \text{ set of exams}, \\
T &: \text{ set of timeslots}, \\
R &: \text{ set of individuals rooms}, \\
G &: \text{ set of room group (super rooms)}, \\
S &: \text{ set of students}, \\
E_s &\subseteq C \text{ exams taken by student } s.
G_r &\subseteq R \text{ room belonging to group} r.
\end{aligned}
$$

## Parameters
$$
\begin{aligned}
dur_i &: \text{ duration of exam } i, \\
len_t &: \text{ length of timeslot } t, \\
cap_r &: \text{ capacity of room } r, \\
Cap_g = \sum_{r \in G_g} cap_r &: \text{super room capacity}, \\
size_i &: \text{ number of students taking exam } i.
\end{aligned}
$$

## Decision variables

### 1. Timeslot assignment
$$
x_{i,t} =
\begin{cases}
1 & \text{if exam } i \text{ is scheduled in timeslot } t, \\
0 & \text{otherwise}.
\end{cases}
$$

### 2. Use of individuals rooms
$$
z_{i,t,r} =
\begin{cases}
1 & \text{if exam } i \text{ uses room } r \text{ in timeslot } t, \\
0 & \text{otherwise}.
\end{cases}
$$

### 3. Use of super rooms (room merging)
$$
y_{i,t,g} =
\begin{cases}
1 & \text{if exam } i \text{ uses super room } g \text{ in timeslot } t, \\
0 & \text{otherwise}.
\end{cases}
$$

# Constraints

## 1. Each exam assigned exactly one timeslot
$$
\sum_{t \in T} x_{i,t} = 1 
\qquad \forall i \in C
$$

## 2. Linking: exam must be in timesplot before using rooms
Individual rooms: 
$$
z_{i,t,r} \le x_{i,t}
\qquad \forall i \in C,\; t \in T,\; r \in R
$$

Super rooms: 
$$
y_{i,t,g} \le x_{i,t}
\qquad \forall i \in C,\; t \in T,\; g \in G
$$

## 3. No student may have overlapping exams
$$
x_{i,t} + x_{j,t} \le 1
\qquad \forall s \in S,\; \forall i \neq j \in E_s,\; t \in T
$$

## 4. Capacity requirement with room merging
Exam $i$ receives capacity from: 
- individual room $r$
- or super room $g$

Total capacity must satisfy exam size:
$$
\sum_{r\in R} cap_r z_{i,t,r} + \sum_{g \in G} Cap_g y_{i,t,g} \geq size_i x_{i,t}
\qquad \forall i \in C, t \in T
$$

## 5. Super room occupancy activities all constituent rooms
If a super room group $g$ is used, then all of its rooms must be marked occupied:
$$
z_{i,t,r} \geq y_{i,t,g}
\qquad \forall g \in G, \forall r \in G_g,\; \forall i, t
$$

This ensures rooms in the group cannot be used by another exam.

## 6. No room can host more than one exam in a timeslot
$$
\sum_{i \in C} z_{i,t,r} \le 1
\qquad \forall t \in T,\; r \in R
$$

## 7. Exam duration feasibility  
$$
x_{i,t} = 0
\qquad \forall i \in C, t \in T \text { such that } dur_i > len_t
$$

# Objective
$$
\min 0
$$

In [16]:
model3 = gp.Model("Exam timetabling with room merging")

# define together groups
together_groups = [
    ["POPE-A13", "POPE-A14"],
    ["ART-LECTURE", "ART-SEMINAR"],
    ["SPORT-LGE1", "SPORT-LGE2"],
]

# Decision variables
# x[e,t] = 1 if exam e is scheduled in timeslot t
x = model3.addVars(exam_list, timeslots, vtype=GRB.BINARY, name="x")

# y[e,r] = aggregated room usage (not used for capacity anymore, but kept for reporting)
y = model3.addVars(exam_list, room_list, vtype=GRB.BINARY, name="y")

# z[e,t,r] = 1 if exam e uses room r in timeslot t
z = model3.addVars(exam_list, timeslots, room_list, vtype=GRB.BINARY, name="z")

# Constraints

# 1. Each exam must be assigned exactly one timeslot
for e in exam_list:
    model3.addConstr(
        gp.quicksum(x[e, t] for t in timeslots) == 1, name=f"timeslot_once_{e}"
    )

# 2. Linking constraint: z[e,t,r] <= x[e,t]
for e in exam_list:
    for t in timeslots:
        for r in room_list:
            model3.addConstr(z[e, t, r] <= x[e, t], name=f"link_zx_{e}_{t}_{r}")

# 3. Aggregate y = SUM_t z
for e in exam_list:
    for r in room_list:
        model3.addConstr(
            y[e, r] == gp.quicksum(z[e, t, r] for t in timeslots),
            name=f"aggregate_y_{e}_{r}",
        )

# 4. No student may have overlapping exams
for s, exams_s in students_exams.items():
    for t in timeslots:
        for i in range(len(exams_s)):
            for j in range(i + 1, len(exams_s)):
                e1, e2 = exams_s[i], exams_s[j]
                model3.addConstr(
                    x[e1, t] + x[e2, t] <= 1, name=f"no_overlap_{s}_{e1}_{e2}_{t}"
                )

# 5. NEW CAPACITY CONSTRAINT: multi-room allowed
for e in exam_list:
    for t in timeslots:
        model3.addConstr(
            gp.quicksum(cap[r] * z[e, t, r] for r in room_list) >= size[e] * x[e, t],
            name=f"capacity_fixed_{e}_{t}",
        )

# 6. Duration feasibility
for e in exam_list:
    for t in timeslots:
        if duration[e] > slot_length[t]:
            model3.addConstr(x[e, t] == 0, name=f"duration_forbidden_{e}_{t}")

# 7. Room conflict: a room can host at most one exam per timeslot
for r in room_list:
    for t in timeslots:
        model3.addConstr(
            gp.quicksum(z[e, t, r] for e in exam_list) <= 1,
            name=f"room_conflict_{r}_{t}",
        )

# 8. Together-group constraints
for group in together_groups:
    base_room = group[0]
    for r in group[1:]:
        for e in exam_list:
            for t in timeslots:
                model3.addConstr(
                    z[e, t, base_room] == z[e, t, r],
                    name=f"together_{e}_{t}_{base_room}_{r}",
                )

# Objective
model3.setObjective(0, GRB.MINIMIZE)

# Optimize
model3.optimize()

print("Model 3: Exam timetabling with room merging (updated)")
for e in exam_list:
    assigned_t = [t for t in timeslots if x[e, t].X > 0.5][0]
    assigned_rooms = [r for r in room_list if y[e, r].X > 0.5]
    print(f"Exam {e}: Timeslot {assigned_t}, Rooms {assigned_rooms}")

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2459974 rows, 446880 columns and 6129288 nonzeros
Model fingerprint: 0x30f2c3d5
Variable types: 0 continuous, 446880 integer (446880 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2022685 rows and 104780 columns
Presolve time: 1.82s
Presolved: 437289 rows, 342100 columns, 1919064 nonzeros
Variable types: 0 continuous, 342100 integer (342100 binary)
Performing another presolve...
Presolve time: 1.75s
Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.03s

Barrier statistics:
 AA' NZ     : 3.128e+05
 Factor NZ  : 9.435e+05 (roughly 26 MB of memory)
 Factor 

AttributeError: Unable to retrieve attribute 'X'

In [ ]:
model3 = gp.Model("Exam timetabling with room merging")

# Super-room definitions
together_groups = [
    ["POPE-A13", "POPE-A14"],
    ["ART-LECTURE", "ART-SEMINAR"],
    ["SPORT-LGE1", "SPORT-LGE2"],
]

# Precompute super-room capacities
super_rooms = list(range(len(together_groups)))
super_capacity = {g: sum(cap[r] for r in together_groups[g]) for g in super_rooms}

# Map super-room -> list of rooms
super_to_rooms = {g: together_groups[g] for g in super_rooms}

# Decision variables
# x[e,t] = 1 if exam e is assigned to timeslot t
x = model3.addVars(exam_list, timeslots, vtype=GRB.BINARY, name="x")

# z[e,t,r] = 1 if exam e uses room r in timeslot t
z = model3.addVars(exam_list, timeslots, room_list, vtype=GRB.BINARY, name="z")

# y[e,t,g] = 1 if exam e activates super-room g in timeslot t
y = model3.addVars(exam_list, timeslots, super_rooms, vtype=GRB.BINARY, name="y")

# Constraints
# 1. Each exam must be assigned exactly one timeslot
for e in exam_list:
    model3.addConstr(
        gp.quicksum(x[e, t] for t in timeslots) == 1, name=f"timeslot_once_{e}"
    )

# 2. Linking: cannot use room without being in that timeslot
for e in exam_list:
    for t in timeslots:
        for r in room_list:
            model3.addConstr(z[e, t, r] <= x[e, t], name=f"link_zx_{e}_{t}_{r}")

# 3. Student conflict: no overlapping exams
for student, exams_s in students_exams.items():
    for t in timeslots:
        for i in range(len(exams_s)):
            for j in range(i + 1, len(exams_s)):
                e1, e2 = exams_s[i], exams_s[j]
                model3.addConstr(
                    x[e1, t] + x[e2, t] <= 1, name=f"no_overlap_{student}_{e1}_{e2}_{t}"
                )

    # 4. Capacity constraint with super room merging
    for t in timeslots:
        model3.addConstr(
            # Sum individual rooms
            gp.quicksum(cap[r] * z[e, t, r] for r in room_list) +
            # Sum super-room capacities
            gp.quicksum(super_capacity[g] * y[e, t, g] for g in super_rooms)
            >= size[e] * x[e, t],
            name=f"capacity_total_{e}_{t}",
        )

# 5. Super room activation forces all internal rooms to be blocked
for g in super_rooms:
    rooms_in_g = super_to_rooms[g]
    for e in exam_list:
        for t in timeslots:
            for r in rooms_in_g:
                model3.addConstr(
                    z[e, t, r] >= y[e, t, g], name=f"superroom_block_{e}_{t}_{g}_{r}"
                )

# 6. Room conflict: room r can host at most one exam per timeslot
for r in room_list:
    for t in timeslots:
        model3.addConstr(
            gp.quicksum(z[e, t, r] for e in exam_list) <= 1,
            name=f"room_conflict_{r}_{t}",
        )

# 7. Duration feasibility
for e in exam_list:
    for t in timeslots:
        if duration[e] > slot_length[t]:
            model3.addConstr(x[e, t] == 0, name=f"duration_forbidden_{e}_{t}")

# Objective
model3.setObjective(0, GRB.MINIMIZE)

model3.optimize()

print("Model 3: Exam timetabling with room merging \n")
for e in exam_list:
    ts = [t for t in timeslots if x[e, t].X > 0.5][0]
    used_rooms = [r for r in room_list if z[e, ts, r].X > 0.5]
    used_super = [g for g in super_rooms if y[e, ts, g].X > 0.5]
    print(f"Exam {e}: Timeslot {ts}, Rooms {used_rooms}, SuperRooms {used_super}")

## Model 4: Minimizing same day student conflicts

### Additional Set
$$
D: \text{ set of days}
$$

Each timeslot $t \in T$ belongs to exactly one day denoted $\text{day}(t) \in D$.

### New decision variable
$$
c_{s,i,j} =
\begin{cases}
1 & \text{if student } s \text{ has exams } i \text{ and } j \text{ on the same day}, \\
0 & \text{otherwise}.
\end{cases}
$$

(Defined only for pairs $i < j$ where student $s$ is enrolled in both.)

## Constraints

### 7. Define same-day conflict indicator
For all students $s$, for all exam pairs $(i < j \in E_s)$:

$$
c_{s,i,j} \geq \sum_{t:day(t)=d} x_{i,t} + \sum_{t:day(t)=d}x x_{j,t} -1 
\quad \forall d \in D
$$

(This forces $(c_{s,i,j} = 1)$ if the two exams are scheduled on the same day.)

## Objective: Minimize total same-day conflicts

$$
\min \sum_{s \in S} \sum_{i < j \in E_s} c_{s,i,j}
$$

### Notes
- Model 3 **keeps all constraints from Model 1 and Model 2** (feasibility).  
- Model 3 only **adds conflict indicators and minimization objective**.

In [ ]:
model4 = gp.Model("Minimizing same day student conflicts")

# Build student exam pairs (i < j)
student_pairs = {}
for s, exams_s in students_exams.items():
    pairs = []
    for i in range(len(exams_s)):
        for j in range(i + 1, len(exams_s)):
            pairs.append((exams_s[i], exams_s[j]))
    student_pairs[s] = pairs

# Timeslot -> Day mapping
timeslot_to_day = {row["timeslot"]: row["day"] for _, row in times_expanded.iterrows()}

days = sorted(times_expanded["day"].unique())

# Decision variables
# x[e,t] = 1 if exam e is assigned to timeslot t
x = model4.addVars(exam_list, timeslots, vtype=GRB.BINARY, name="x")

# z[e,t,r] = 1 if exam e uses room r in timeslot t
z = model4.addVars(exam_list, timeslots, room_list, vtype=GRB.BINARY, name="z")

# y[e,t,g] = 1 if exam e activates super-room g in timeslot t
y = model4.addVars(exam_list, timeslots, super_rooms, vtype=GRB.BINARY, name="y")

# c[s,i,j] = 1 if exams i,j on same day
c = model4.addVars(
    [(s, i, j) for s, pairs in student_pairs.items() for (i, j) in pairs],
    vtype=GRB.BINARY,
    name="same_day_conflict",
)

# Constraints
# 1. Each exam must be assigned exactly one timeslot
for e in exam_list:
    model4.addConstr(
        gp.quicksum(x[e, t] for t in timeslots) == 1, name=f"timeslot_once_{e}"
    )

# 2. Linking: cannot use room without being in that timeslot
for e in exam_list:
    for t in timeslots:
        for r in room_list:
            model4.addConstr(z[e, t, r] <= x[e, t], name=f"link_zx_{e}_{t}_{r}")

# 3. Student conflict: no overlapping exams
for student, exams_s in students_exams.items():
    for t in timeslots:
        for i in range(len(exams_s)):
            for j in range(i + 1, len(exams_s)):
                e1, e2 = exams_s[i], exams_s[j]
                model4.addConstr(
                    x[e1, t] + x[e2, t] <= 1, name=f"no_overlap_{student}_{e1}_{e2}_{t}"
                )

    # 4. Capacity constraint with super room merging
    for t in timeslots:
        model4.addConstr(
            # Sum individual rooms
            gp.quicksum(cap[r] * z[e, t, r] for r in room_list) +
            # Sum super-room capacities
            gp.quicksum(super_capacity[g] * y[e, t, g] for g in super_rooms)
            >= size[e] * x[e, t],
            name=f"capacity_total_{e}_{t}",
        )

# 5. Super room activation forces all internal rooms to be blocked
for g in super_rooms:
    rooms_in_g = super_to_rooms[g]
    for e in exam_list:
        for t in timeslots:
            for r in rooms_in_g:
                model4.addConstr(
                    z[e, t, r] >= y[e, t, g], name=f"superroom_block_{e}_{t}_{g}_{r}"
                )

# 6. Room conflict: room r can host at most one exam per timeslot
for r in room_list:
    for t in timeslots:
        model4.addConstr(
            gp.quicksum(z[e, t, r] for e in exam_list) <= 1,
            name=f"room_conflict_{r}_{t}",
        )

# 7. Duration feasibility
for e in exam_list:
    for t in timeslots:
        if duration[e] > slot_length[t]:
            model4.addConstr(x[e, t] == 0, name=f"duration_forbidden_{e}_{t}")

# 8. Activate c if exams fall on the same day
for s, pairs in student_pairs.items():
    for i, j in pairs:
        for d in days:
            t_day = [t for t in timeslots if timeslot_to_day[t] == d]

            model4.addConstr(
                c[s, i, j]
                >= gp.quicksum(x[i, t] for t in t_day)
                + gp.quicksum(x[j, t] for t in t_day)
                - 1,
                name=f"same_day_{s}_{i}_{j}_{d}",
            )


# Objective
model4.setObjective(gp.quicksum(c[key] for key in c.keys()), GRB.MINIMIZE)

model4.optimize()

print("Model 4: Minimizing same day student conflicts \n")
for e in exam_list:
    t_assigned = next(t for t in timeslots if x[e, t].X > 0.5)
    rooms_used = [r for r in room_list if z[e, t_assigned, r].X > 0.5]
    print(f"Exam {e} -> Timeslot {t_assigned}, Rooms {rooms_used}")

## Model 5: Minimizing consecutive or overnight exam conflicts

### Additional Definition
Each timeslot $t \in T$ has an associated ordering index $\text{ord}(t)$ such that \( \text{ord}(t+1) \) is the next available timeslot.

A pair of exams is considered consecutive for a student if they occur in timeslots $t$ and $t'$ with $\text{ord}(t') = \text{ord}(t) + 1$.

### New decision variable
$$
o_{s,i,j} =
\begin{cases}
1 & \text{if student } s \text{ has exams } i \text{ and } j \text{ in consecutive timeslots}, \\
0 & \text{otherwise}.
\end{cases}
$$

# New constraints for model 5

### 8. Consecutive timeslot conflict indicator
For all students $s$, exams $i < j \in E_s$, and all consecutive slots $t, t'$:

$$
o_{s,i,j} \ge x_{i,t} + x_{j,t'} - 1
\qquad \forall s,\, \forall i<j \in E_s,\, \forall (t,t') \text{ such that } \text{ord}(t') = \text{ord}(t)+1.
$$

Symmetrically, the reverse order is also a conflict:

$$
o_{s,i,j} \ge x_{i,t'} + x_{j,t} - 1
\qquad \forall s,\, \forall i<j \in E_s,\, \forall (t,t') \text{ consecutive}.
$$

## Objective for model 5
Minimize the number of consecutive or overnight exam conflicts:

$$
\min \sum_{s \in S} \sum_{i<j \in E_s} o_{s,i,j}
$$

## Model 6: Institutional constraints and priority scheduling


Model 6 extends the previous models by incorporating institutional, departmental, and logistical rules extracted from the University of Nottingham dataset. No new decision variables are added. All constraints below apply to the existing variables:


$$
x_{i,t} =
\begin{cases}
1 & \text{if exam $i$ is scheduled in timeslot $t$,} \\
0 & \text{otherwise,}
\end{cases}
\qquad
y_{i,r} =
\begin{cases}
1 & \text{if exam $i$ is assigned room $r$,} \\
0 & \text{otherwise.}
\end{cases}
$$


Let:
$A_i \subseteq T$ be allowed timeslots for exam $i$.


$R_i \subseteq R$ be allowed rooms for exam $i$.


$P_i$ = priority weight for earliness.


$\mathrm{ord}(t)$ be chronological index of timeslot $t$.


$\mathcal{G}$ be set of standard coincidence groups.


$\mathcal{G}^{\mathrm{seq}}$ be special coincidence groups requiring ordered execution within a single long timeslot.


$\mathcal{G}^{\mathrm{one}}$ be coincidence groups required to share exactly one room and exclude all other exams.


$\mathcal{S}$ be set of exam pairs $(i,j)$ where $i$ must occur before $j$.


$\mathcal{F}$ be set of forbidden room–timeslot pairs $(r,t)$.


$\mathcal{D}^{\mathrm{AM}},\ \mathcal{D}^{\mathrm{PM}}$ be AM-only or PM-only exam sets.


$\mathcal{X}$ be set of spread-out exam groups.


$\mathcal{R}^{\mathrm{comb}}_i$ be pairs of rooms $\,(r,r')$ that must be simultaneously assigned to exam $i$.


## Constraint


### 1. Allowed timeslot constraints


$$
x_{i,t} = 0 \qquad \forall i \in C,\ \forall t \notin A_i.
$$


This encodes restrictions such as:


$K1AHWAE2$: allowed only in AM slots.


$V13101E1$: allowed only in Thursday PM slots.


Exams restricted to the calendar window Jan 23--24.




### 2. Allowed room constraints


$$
y_{i,r} = 0 \qquad \forall i \in C,\ \forall r \notin R_i.
$$


Examples encoded in $R_i$ include:


ART exams restricted to **ART-LECTURE** or **ART-SEMINAR** rooms.


Chemistry exams requiring **CHEMISTRY** laboratories.


Architecture exams requiring **ARCHTCT-LR1**.


$AA3008E1$ requiring the special two-phase assignment (see sequencing constraints below).


### 3. Standard coincidence constraints


For every coincidence group $G \in \mathcal{G}$,all exams must occur in the same timeslot:
$$
x_{i,t} = x_{j,t}
\qquad \forall G\in\mathcal{G},\ \forall i,j \in G,\ \forall t\in T.
$$


These include groups such as:
$$
\{ \text{LK44FAE1, LK44GAE1, LK44SAE1} \},
\quad
\{ \text{M11341E1, M11345E1} \},
\quad \text{etc.}
$$


### 4. Special ``one-after-the-other'' coincidence groups


For any group $G \in \mathcal{G}^{\mathrm{seq}}$ 
(e.g.\ $\{\text{C13563E1}, \text{C13571E1}, \text{C13572E1}\}$),
all exams must share the same timeslot:
$$
x_{i,t} = x_{j,t}
\qquad \forall G\in\mathcal{G}^{\mathrm{seq}},\ \forall i,j\in G,\ \forall t.
$$


Additionally, exams that must occur in order (back-to-back within the same slot)
satisfy:
$$
\mathrm{orderWithinSlot}(i) < \mathrm{orderWithinSlot}(j)
\qquad \forall (i,j) \in \mathcal{G}^{\mathrm{seq}}_{\mathrm{ordered}}.
$$


(If the model treats timeslots as atomic, this may be relaxed.)


### 5. One-room-only coincidence groups


For groups $G \in \mathcal{G}^{\mathrm{one}}$
(e.g.\ $\{\text{H22C20E1}, \text{H23C20E1}, \text{H24C20E1}, \text{H23CEOE1}\}$):


$$
x_{i,t} = x_{j,t}
\qquad \forall i,j\in G,\ \forall t.
$$


All exams must share exactly one room:
$$
y_{i,r} = y_{j,r}
\qquad \forall i,j\in G,\ \forall r\in R.
$$


No other exam may use that room:
$$
\sum_{k \notin G} y_{k,r} = 0
\qquad \forall r \text{ with } y_{i,r} = 1.
$$


### 6. Sequencing constraints (before/after)


For each ordered pair $(i,j) \in \mathcal{S}$:
$$
\sum_{t\in T} \mathrm{ord}(t)\, x_{i,t}
<
\sum_{t\in T} \mathrm{ord}(t)\, x_{j,t}.
$$


Examples include:
$$
\text{F13P03E1} \prec \text{F13X03E1},
\qquad
\text{F13P05E1} \prec \text{F13X04E1},
\qquad
\text{H3BFM2E1} \text{ immediately before } \text{H3BFM2E2}.
$$


### 7. Room unavailability and forbidden pairs


For each forbidden room--timeslot pair $(r,t)\in\mathcal{F}$,
such as TRENT-B46 on Friday AM:
$$
y_{i,r} + x_{i,t} \le 1
\qquad \forall i\in C.
$$


\subsubsection*{8. AM-only and PM-only exam constraints}


$$
x_{i,t} = 0
\qquad \forall i \in \mathcal{D}^{\mathrm{AM}},\ \forall t \notin \mathrm{AM}.
$$


$$
x_{i,t} = 0
\qquad \forall i \in \mathcal{D}^{\mathrm{PM}},\ \forall t \notin \mathrm{PM}.
$$


### 9. Spread-out constraints


For each spread-out group $X\in\mathcal{X}$,
exams must not occur too close together. For every $i,j\in X$:
$$
\lvert\, \mathrm{ord}(t_i) - \mathrm{ord}(t_j) \,\rvert \ge \Delta_X,
$$
where
$$
t_i = \sum_{t\in T} \mathrm{ord}(t)\, x_{i,t}.
$$


Groups include:
$$
\{\text{Q33211E1},\ \text{Q33307E1},\ \text{Q33308E1}\},
\qquad
\{\text{B12301E1},\text{B12302E1},\text{B12303E1},\text{B12320E1}\},
\quad \text{etc.}
$$


### 10. Combined-room requirements


If exam $i$ must use both rooms $r$ and $r'$ simultaneously:
$$
y_{i,r} = y_{i,r'}
\qquad \forall (r,r') \in \mathcal{R}^{\mathrm{comb}}_i.
$$


This includes exams requiring POPE-A13 and POPE-A14 jointly.


### 11. Earliness penalty


Define:
$$
\mathrm{penalty}_i
=
\sum_{t\in T} P_i \cdot \mathrm{ord}(t) \cdot x_{i,t}.
$$


### 12.  Model 5 Objective


$$
\min
\left(
\alpha \sum_{s,i,j} c_{s,i,j}
\;+\;
\beta \sum_{s,i,j} o_{s,i,j}
\;+\;
\gamma \sum_{i\in C} \mathrm{penalty}_i
\right).
$$


Where:


$c_{s,i,j}$ = consecutive-conflict indicator,


$o_{s,i,j}$ = same-day conflict indicator,


$\alpha,\beta,\gamma$ = user-defined weights.